# Phoenix Households by AMI Band - PUMS data
### specify year, import AMI bands in py file

- https://www.census.gov/data/developers/data-sets/

For households by income and household size to compare to HUD AMI in same year
-  https://api.census.gov/data/2021/acs/acs1/pums/variables.html

2021 AMI by HH Size (from City of Phoenix)
- https://www.phoenix.gov/humanservicessite/Documents/2021%20AMI%20Limits%204.2.21.pdf

2012 AMI by HH Size (from AZ Housing)
- https://housing.az.gov/sites/default/files/documents/files/2012%2520SHF-CDBG%2520Med%2520Income%2520Limits.pdf

In [1]:
import pandas as pd
import math
import numpy as np
import os

In [2]:
import get_pums as get
import pums as calc

In [3]:
#from AMI_BANDS_2021 import *
from AMI_BANDS_2012 import *

In [4]:
#Search parameters
#year = '2021'
year = '2012'

sample = 'acs1'

phx_pumas = ['0400112','0400113','0400114','0400115','0400116','0400117',
             '0400118','0400119','0400120','0400121','0400122','0400123',
             '0400125','0400128','0400129']

data_cols = 'SERIALNO,ST,PUMA,HINCP,NP,WGTP,ADJINC'

In [5]:
# create a list of replicate weights
repwt = 'WGTP'
repwts = [repwt+str(i) for i in range(1, 81)]

## Get PUMA data

In [6]:
raw = get.get_puma(sample,year,data_cols)

In [7]:
raw.head()

,SERIALNO,ST,PUMA,HINCP,NP,WGTP,ADJINC,WGTP1,WGTP2,WGTP3,...,WGTP71,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80
0,140,4,101,-60000,0,202,1010207,206,206,359,...,55,203,189,369,211,204,199,206,340,194
1,152,4,204,57700,2,47,1010207,53,13,84,...,46,50,46,13,51,15,47,46,17,13
2,285,4,113,327000,5,111,1010207,194,102,34,...,148,170,194,33,112,121,194,38,114,33
3,365,4,104,-60000,1,0,1010207,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,538,4,209,-60000,1,0,1010207,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df = raw.copy()

In [9]:
df['ST'] = df['ST'].str.zfill(2)
df['PUMA'] = df['PUMA'].str.zfill(5)
df['GEO_ID'] = df['ST']+df['PUMA']

In [10]:
#make geo identifiers, reduce df to PHX PUMAs
df['GEO_ID'] = df['ST']+df['PUMA']
df = df[df.GEO_ID.isin(phx_pumas)]
df  = df.drop(['SERIALNO','ST','PUMA'],axis=1)
df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']] #move id to first col

#clean data cols to be numeric
for col in df.columns[1:]: df[col] = df[col].astype(float)

In [11]:
df['HHSz'] = pd.cut(df['NP'],bins=[0,1,2,3,4,5,6,7,14],
                   labels=['1','2','3','4','5','6','7','8'])

In [12]:
dff = df[~(df.HHSz.isna()) & (df.HINCP!=-60000)].copy()

#generate adjusted household income using ACS adjustment factor
if year == '2012': 
    df['HINCP'] = df.ADJINC/1000000 * df.HINCP
else:
    df['HINCP'] = df.ADJINC * df.HINCP

print(dff.WGTP.sum())

dff['HHSz'] = dff['HHSz'].astype(str)

570939.0


In [13]:
#assign AMI range based on household size and ami dictionaries
dff['AMI_range'] = np.where((dff['HINCP']<=dff['HHSz'].map(AMI_30pct)),inc_lbl[0],
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_30pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_50pct)),inc_lbl[1],
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_50pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_80pct)),
                   inc_lbl[2],np.where((dff['HINCP']>dff['HHSz'].map(AMI_80pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_100pct)),
                   inc_lbl[3],np.where((dff['HINCP']>dff['HHSz'].map(AMI_100pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_120pct)),
                                       inc_lbl[4],inc_lbl[5])))))

In [14]:
pivot = dff.drop(['HINCP','NP','HHSz'],axis=1).copy()
pivot = pivot.groupby(['GEO_ID','AMI_range']).sum().reset_index()
pivot.rename(columns={'WGTP':'hh_e'},inplace=True)

In [15]:
pivot['hh_SE'] = pivot.apply(lambda x: (calc.get_se(x['hh_e'],x[repwts])),axis=1)
pivot['hh_MOE'] = pivot.apply(lambda x: (calc.get_moe(x['hh_SE'])),axis=1)
pivot['hh_CV'] = pivot.apply(lambda x: (calc.get_cv(x['hh_e'],x['hh_SE'])),axis=1)
pivot = pivot.drop(columns=repwts)

In [16]:
final = pd.pivot_table(pivot,values=['hh_e','hh_MOE','hh_CV'],index='GEO_ID',\
                      columns=['AMI_range'],aggfunc=np.sum).reset_index()

In [17]:
final.head(10)

GEO_ID       hh_CV                                           \
AMI_range          100_120_ami 30_50_ami 50_80_ami 80_100_ami  o120_ami   
0          0400112    9.981710  8.646760  8.501021  10.091412  2.209167   
1          0400113    8.144682  8.522212  6.431949   7.709189  3.090106   
2          0400114    6.907018  6.301979  6.474296   7.221352  3.852305   
3          0400115    7.534973  6.734079  5.025609   8.410747  3.839741   
4          0400116    8.339828  6.720268  4.516386   7.798825  3.623913   
5          0400117    8.817442  5.542496  5.327219   6.344080  3.460429   
6          0400118    9.339991  4.149375  5.557637   8.678641  4.506186   
7          0400119   11.406509  6.222621  5.268856   8.180146  5.258375   
8          0400120    8.013873  9.238619  7.045370  10.701269  2.495368   
9          0400121    9.974906  8.444552  5.161696  10.895987  5.122781   

                           hh_MOE                                         \
AMI_range    u30_ami  100_120_ami    30_50_ami    50_80_ami   80_100_ami   
0          11.413528   791.145045   603.911348   970.997811   537.413846   
1           7.931458   829.354366   785.237868  1132.370225   871.792035   
2           6.409176   708.746203   855.564682  1347.082580   896.352975   
3           5.531907   764.006787  1259.727317  1299.968965  1084.499358   
4           5.562832   839.070222   952.541508  1100.541914   693.260313   
5           4.028254   822.700378   785.002247  1031.579520   757.418542   
6           4.046981   667.998377   835.386409  1023.711797   848.499268   
7           4.537349   473.797696   732.140880  1017.711527   780.946750   
8           9.878480  1006.001562   758.997991   885.376189  1071.152806   
9           5.698256   737.970725   851.206001  1289.077068  1537.927867   

                                           hh_e                      \
AMI_range     o120_ami      u30_ami 100_120_ami 30_50_ami 50_80_ami   
0          1842.916505   668.048860      2929.0    2581.0    4221.0   
1          1727.820251   840.265654      3763.0    3405.0    6506.0   
2          1829.175223   912.262407      3792.0    5017.0    7689.0   
3          1055.148702   779.460732      3747.0    6913.0    9559.0   
4          1147.544678  1384.589865      3718.0    5238.0    9005.0   
5          1192.131909  1112.074781      3448.0    5234.0    7156.0   
6          1163.171637  1131.700373      2643.0    7440.0    6807.0   
7           765.962901   864.260989      1535.0    4348.0    7138.0   
8          1387.644501   761.043343      4639.0    3036.0    4644.0   
9           988.248554  1284.762956      2734.0    3725.0    9229.0   

                                        
AMI_range 80_100_ami o120_ami  u30_ami  
0             1968.0  30828.0   2163.0  
1             4179.0  20663.0   3915.0  
2             4587.0  17547.0   5260.0  
3             4765.0  10155.0   5207.0  
4             3285.0  11702.0   9198.0  
5             4412.0  12731.0  10202.0  
6             3613.0   9539.0  10334.0  
7             3528.0   5383.0   7039.0  
8             3699.0  20550.0   2847.0  
9             5216.0   7129.0   8332.0

In [18]:
final.to_excel(f'../../output/acs/hhinc_120ami_pumas_{year[-2:]}.xlsx')